In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
data_text=data[['headline_text']]
data_text['index']=data_text.index
documents=data_text

In [3]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [4]:
print(len(documents))

1103663


In [8]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [12]:
import numpy as np
np.random.seed(2018)

In [27]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /Users/ling/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ling/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [40]:
# def lemmatize_stemming(text):
#     lemmatizer = nltk.WordNetLemmatizer()
# #     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
#     processed_tokens = nltk.word_tokenize(text)
#     processed_tokens = [w.lower() for w in processed_tokens]
#     for w in processed_tokens:
#         processed_tokens.append(lemmatizer.lemmatize(w)) 
#     return processed_tokens

def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

In [41]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [42]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfires']


In [43]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0              [decide, community, broadcast, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4             [strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                         [australia, lock, timetable]
Name: headline_text, dtype: object

In [44]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [45]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [46]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(112, 1), (489, 1), (970, 1), (4427, 1)]

In [47]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

Word 112 ("help") appears 1 time.
Word 489 ("rain") appears 1 time.
Word 970 ("bushfires") appears 1 time.
Word 4427 ("dampen") appears 1 time.


## LDA using Bag of Words

In [48]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.595557507312157),
 (1, 0.3948343404468723),
 (2, 0.4976240764382213),
 (3, 0.4917187993528624)]


In [49]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [50]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face" + 0.016*"jail" + 0.016*"years" + 0.015*"shoot" + 0.015*"accuse" + 0.015*"drug"
Topic: 1 
Words: 0.020*"market" + 0.014*"share" + 0.012*"fall" + 0.011*"news" + 0.010*"port" + 0.010*"build" + 0.009*"industry" + 0.009*"marriage" + 0.009*"energy" + 0.009*"say"
Topic: 2 
Words: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal" + 0.012*"return" + 0.011*"abuse" + 0.010*"say" + 0.009*"young" + 0.009*"china"
Topic: 3 
Words: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women" + 0.012*"flood" + 0.012*"bank" + 0.012*"guilty" + 0.011*"rise" + 0.011*"royal"
Topic: 4 
Words: 0.031*"queensland" + 0.023*"coast" + 0.020*"live" + 0.015*"claim" + 0.015*"gold" + 0.014*"north" + 0.012*"force" + 0.011*"green" + 0.011*"fear" + 0.011*"party"
Topic: 5 
Words: 0.055*"police" + 0.022*"kill" + 0.021*"crash" + 0.018*"die" + 0.017*"interview" + 0.016*"miss" + 0

In [52]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4199962615966797	 
Topic: 0.015*"plan" + 0.014*"health" + 0.014*"fund" + 0.014*"council" + 0.012*"help" + 0.012*"water" + 0.011*"say" + 0.009*"service" + 0.009*"park" + 0.009*"budget"

Score: 0.22000005841255188	 
Topic: 0.044*"australia" + 0.023*"election" + 0.021*"south" + 0.014*"test" + 0.013*"power" + 0.013*"tasmania" + 0.012*"people" + 0.012*"turnbull" + 0.009*"island" + 0.009*"violence"

Score: 0.21999981999397278	 
Topic: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face" + 0.016*"jail" + 0.016*"years" + 0.015*"shoot" + 0.015*"accuse" + 0.015*"drug"

Score: 0.020002376288175583	 
Topic: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal" + 0.012*"return" + 0.011*"abuse" + 0.010*"say" + 0.009*"young" + 0.009*"china"

Score: 0.020001482218503952	 
Topic: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women" + 0.012*"flood" + 0.012*"bank" + 0.012*"guilty" + 0.011*"rise" + 0.011*"royal"

S

In [54]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.35000014305114746	 Topic: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal"
Score: 0.1833333820104599	 Topic: 0.017*"state" + 0.017*"country" + 0.016*"rural" + 0.014*"hospital" + 0.012*"change"
Score: 0.18333332240581512	 Topic: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face"
Score: 0.18333317339420319	 Topic: 0.044*"australia" + 0.023*"election" + 0.021*"south" + 0.014*"test" + 0.013*"power"
Score: 0.01666666753590107	 Topic: 0.020*"market" + 0.014*"share" + 0.012*"fall" + 0.011*"news" + 0.010*"port"
Score: 0.01666666753590107	 Topic: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women"
Score: 0.01666666753590107	 Topic: 0.031*"queensland" + 0.023*"coast" + 0.020*"live" + 0.015*"claim" + 0.015*"gold"
Score: 0.01666666753590107	 Topic: 0.055*"police" + 0.022*"kill" + 0.021*"crash" + 0.018*"die" + 0.017*"interview"
Score: 0.01666666753590107	 Topic: 0.020*"canberra" + 0.020*"open" + 0.015

## LDA using TF-IDF

In [51]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.019*"rural" + 0.014*"crash" + 0.008*"royal" + 0.007*"fatal" + 0.006*"die" + 0.006*"truck" + 0.005*"commission" + 0.005*"road" + 0.004*"police" + 0.004*"rail"
Topic: 1 Word: 0.023*"country" + 0.020*"hour" + 0.007*"christmas" + 0.007*"mother" + 0.005*"live" + 0.005*"history" + 0.005*"politics" + 0.005*"wrap" + 0.005*"bail" + 0.005*"prison"
Topic: 2 Word: 0.008*"marriage" + 0.007*"friday" + 0.007*"wednesday" + 0.006*"asylum" + 0.006*"august" + 0.006*"great" + 0.005*"capital" + 0.005*"wild" + 0.004*"children" + 0.004*"allegedly"
Topic: 3 Word: 0.010*"news" + 0.008*"health" + 0.007*"budget" + 0.007*"business" + 0.006*"fund" + 0.006*"say" + 0.005*"sport" + 0.005*"national" + 0.005*"victorian" + 0.005*"monday"
Topic: 4 Word: 0.007*"john" + 0.007*"october" + 0.007*"september" + 0.005*"outback" + 0.004*"scott" + 0.004*"footage" + 0.004*"refugees" + 0.004*"speed" + 0.003*"young" + 0.003*"knight"
Topic: 5 Word: 0.009*"australia" + 0.007*"world" + 0.007*"grandstand" + 0.006*"abbot

In [53]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.2947407364845276	 
Topic: 0.012*"coast" + 0.009*"turnbull" + 0.009*"weather" + 0.007*"queensland" + 0.007*"search" + 0.007*"flood" + 0.006*"north" + 0.006*"miss" + 0.006*"south" + 0.006*"november"

Score: 0.289143830537796	 
Topic: 0.008*"marriage" + 0.007*"friday" + 0.007*"wednesday" + 0.006*"asylum" + 0.006*"august" + 0.006*"great" + 0.005*"capital" + 0.005*"wild" + 0.004*"children" + 0.004*"allegedly"

Score: 0.27610644698143005	 
Topic: 0.019*"rural" + 0.014*"crash" + 0.008*"royal" + 0.007*"fatal" + 0.006*"die" + 0.006*"truck" + 0.005*"commission" + 0.005*"road" + 0.004*"police" + 0.004*"rail"

Score: 0.020002324134111404	 
Topic: 0.010*"news" + 0.008*"health" + 0.007*"budget" + 0.007*"business" + 0.006*"fund" + 0.006*"say" + 0.005*"sport" + 0.005*"national" + 0.005*"victorian" + 0.005*"monday"

Score: 0.020001716911792755	 
Topic: 0.007*"john" + 0.007*"october" + 0.007*"september" + 0.005*"outback" + 0.004*"scott" + 0.004*"footage" + 0.004*"refugees" + 0.004*"speed" + 0.

In [55]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.359101802110672	 Topic: 0.055*"police" + 0.022*"kill" + 0.021*"crash" + 0.018*"die" + 0.017*"interview"
Score: 0.3326616883277893	 Topic: 0.020*"canberra" + 0.020*"open" + 0.015*"australian" + 0.013*"world" + 0.013*"league"
Score: 0.19155389070510864	 Topic: 0.032*"charge" + 0.030*"court" + 0.030*"sydney" + 0.022*"murder" + 0.020*"face"
Score: 0.01667175441980362	 Topic: 0.021*"home" + 0.015*"tasmanian" + 0.014*"high" + 0.014*"price" + 0.013*"women"
Score: 0.016671014949679375	 Topic: 0.044*"australia" + 0.023*"election" + 0.021*"south" + 0.014*"test" + 0.013*"power"
Score: 0.016668202355504036	 Topic: 0.036*"trump" + 0.031*"government" + 0.016*"donald" + 0.013*"hour" + 0.013*"deal"
Score: 0.016668176278471947	 Topic: 0.031*"queensland" + 0.023*"coast" + 0.020*"live" + 0.015*"claim" + 0.015*"gold"
Score: 0.016667908057570457	 Topic: 0.017*"state" + 0.017*"country" + 0.016*"rural" + 0.014*"hospital" + 0.012*"change"
Score: 0.01666787639260292	 Topic: 0.020*"market" + 0.014*"sha